## TODO
* Maybe they return cashed answers and don't count our requests? Then we'll not exceed requests limit

### Experiment #1: separating block of account from block of ip
#### At the moment, in case of access error we don't know whether account, ip or both were blocked. So we reach limit, and try to use them separately. This way we can find that one of the resources was not blocked and use it further.

In [1]:
import json


with open("../../resources/checkpoints/data_checkpoint.json") as f:
    user_data = json.load(f)

In [3]:
# will just retrieve groups walls
groups_to_get = user_data["68076353"]["groups"]

In [10]:
proxy = input("Proxy address\n")
login = input("Login\n")
password = input("Password\n")

Proxy address
 45.95.99.226:7786
Login
 79082615141
Password
 EITHHzn9


In [11]:
import sys
sys.path.insert(0, "../../")

In [12]:
from suvec.vk_api_impl.session.session_manager import SessionManager
from suvec.vk_api_impl.errors_handler import VkApiErrorsHandler
from suvec.vk_api_impl.session.records_managing import ProxyManager, CredsManager
from suvec.vk_api_impl.session.records_managing.terminal_out_of_records import TerminalOutOfProxy, TerminalOutOfCreds

import utils
from suvec.vk_api_impl.crawl_runner_with_checkpoints import VkCrawlRunnerWithCheckpoints
from suvec.common.events_tracking.terminal_events_tracker import TerminalEventsTracker
from suvec.vk_api_impl.session.records_managing.records_storing import ProxyStorage, CredsStorage
from suvec.vk_api_impl.session.records_managing.records_storing.serializers import ProxyRecordsSerializer, \
    CredsRecordsSerializer

In [13]:

settings_path = "../../settings.json"
proxies_save_pth, creds_save_pth = utils.get_proxy_and_creds_paths(settings_path)
checkp_data, checkp_requester = utils.get_data_requester_checkpoint_paths(settings_path)
result_file = utils.get_result_path(settings_path)
backups_path = utils.get_backups_path(settings_path)

proxy_storage = ProxyStorage(proxies_save_pth, ProxyRecordsSerializer())
creds_storage = CredsStorage(creds_save_pth, CredsRecordsSerializer())


In [14]:
errors_handler = VkApiErrorsHandler(None)

out_of_proxy_handler = TerminalOutOfProxy()
out_of_creds_handler = TerminalOutOfCreds()
proxy_manager = ProxyManager(proxy_storage, None, out_of_proxy_handler,
                             hours_for_resource_reload=24)
creds_manager = CredsManager(creds_storage, None, out_of_creds_handler,
                             hours_for_resource_reload=24)

session_manager = SessionManager(errors_handler, proxy_manager, creds_manager)

check record is usable <suvec.vk_api_impl.session.records_managing.records.ProxyRecord object at 0x7f25f9026c10> 49424.800080776215
check record is usable <suvec.vk_api_impl.session.records_managing.records.ProxyRecord object at 0x7f25f9026820> 44677.80105781555
check record is usable <suvec.vk_api_impl.session.records_managing.records.ProxyRecord object at 0x7f25f8f9e550> 43914.379513025284
check record is usable <suvec.vk_api_impl.session.records_managing.records.ProxyRecord object at 0x7f25f8f9e670> 50659.11779284477
check record is usable <suvec.vk_api_impl.session.records_managing.records.CredsRecord object at 0x7f25f8f9eca0> 49424.8032207489
check record is usable <suvec.vk_api_impl.session.records_managing.records.CredsRecord object at 0x7f25f8f9ed90> 44677.806616306305
check record is usable <suvec.vk_api_impl.session.records_managing.records.CredsRecord object at 0x7f25f8f9ee20> 43914.38683056831
check record is usable <suvec.vk_api_impl.session.records_managing.records.CredsR

In [15]:
session = session_manager._create_session(login, password, proxy)

In [17]:
from time import sleep
from random import choice

cnt_requests = 0

def _make_request(group_id):
    request = session.vk_session.method("groups.getById", values={"group_ids": group_id})
    return request

while True:
    if cnt_requests % 100 == 0:
        print("cnt", cnt_requests)
    for _ in range(3):
        random_group_id = choice(groups_to_get)
        res = _make_request(random_group_id)
        cnt_requests += 1
    sleep(0)

cnt 0
cnt 300
cnt 600
cnt 900
cnt 1200
cnt 1500
cnt 1800
cnt 2100
cnt 2400
cnt 2700
cnt 3000
cnt 3300
cnt 3600
cnt 3900
cnt 4200
cnt 4500
cnt 4800
cnt 5100
cnt 5400
cnt 5700
cnt 6000
cnt 6300
cnt 6600
cnt 6900
cnt 7200
cnt 7500
cnt 7800
cnt 8100
cnt 8400
cnt 8700
cnt 9000
cnt 9300
cnt 9600
cnt 9900
cnt 10200
cnt 10500


ApiError: [29] Rate limit reached

# Blocked ~ at 15:30 11.06.21

In [ ]:
print(res)

### Changed proxy: didn't help, so creds were blocked

In [19]:
other_proxy = "45.95.99.20:7580"
session_other_proxy = session_manager._create_session(login, password, other_proxy)

In [20]:
cnt_requests = 0

def _make_request(group_id):
    request = session_other_proxy.vk_session.method("groups.getById", values={"group_ids": group_id})
    return request

while True:
    if cnt_requests % 100 == 0:
        print("cnt", cnt_requests)
    for _ in range(3):
        random_group_id = choice(groups_to_get)
        res = _make_request(random_group_id)
        cnt_requests += 1
    sleep(0)

cnt 0


ApiError: [29] Rate limit reached

### Changed creds: helped, so proxy were not blocked
### Update: creds were blocked one more time, I reset them and everything works. So the conclusion is: accounts are blocked far more early then proxy

### Note: every cell worked ~ 2 hours before access error, so maybe situation changes in case of hour limits
### Update: works one more time
### Update: at 01:30 reseted creds with ones that exceeded limit at 15:30. Maybe they aren't blocked for 24 hours, blocked for 1 hour, or till the end of the day
## TODO: exceed limit and retry with same creds after hour, after 2 hours etc.

In [28]:
other_creds = "79082615141", "EITHHzn9"
session_other_creds = session_manager._create_session(*other_creds, proxy)

In [29]:
cnt_requests = 0

def _make_request(group_id):
    request = session_other_creds.vk_session.method("groups.getById", values={"group_ids": group_id})
    return request

while True:
    if cnt_requests % 100 == 0:
        print("cnt", cnt_requests)
    for _ in range(3):
        random_group_id = choice(groups_to_get)
        res = _make_request(random_group_id)
        cnt_requests += 1
    sleep(0)

cnt 0
cnt 300
cnt 600
cnt 900
cnt 1200
cnt 1500
cnt 1800
cnt 2100
cnt 2400
cnt 2700
cnt 3000
cnt 3300
cnt 3600
cnt 3900
cnt 4200
cnt 4500
cnt 4800
cnt 5100
cnt 5400
cnt 5700
cnt 6000
cnt 6300
cnt 6600
cnt 6900
cnt 7200
cnt 7500
cnt 7800
cnt 8100
cnt 8400
cnt 8700
cnt 9000
cnt 9300
cnt 9600
cnt 9900
cnt 10200
cnt 10500


ApiError: [29] Rate limit reached